# Exploring the Baseball Databank with pandas

In this notebook we will use the very powerful pandas package to examine the Baseball Databank.

First we need to import the usual packages.

In [ ]:
import numpy as np               #standard imports:  numpy, scipy, pandas, matplotlib
import scipy as sc
import pandas as pd
import matplotlib.pyplot as plt
                                 #the command below allows us to capture matplotlib output in this notebook
%matplotlib inline

### One-time Setup

Here are instructions for downloading the Baseball Databank.

Like so many things these days, the most up-to-date version is on Github.  

Open a command promt and cd into the directory you have been issuing your "git clone" commands from

Once you are in that directory, type:

git clone https://github.com/chadwickbureau/baseballdatabank.git

This will create a subdirectory called "baseballdatabank" that contains the most recent version.

We should be able to access the files using a relative path:   ../../baseballdatabank/core

This says "go up two directory levels from where we are, then look in the "core" subdirectory of the "baseballdatabank" directory.

You can issue commands as if they were typed in at a terminal window with the "!" operator.

In [ ]:
! ls -l ../../baseballdatabank/core

If you are on windows and this doesn't work, try this instead:

  !  dir ..\baseballdatabank\core

## Creating a pandas DataFrame from a .csv file - very easy!

The read_csv() function in pandas is the go-to method if your data is in comma separated values (.csv) format.

The .csv files in this collection have the column names in the first row, which is often the case (but not always).

Now we'll read the Master.csv file using a relative path to locate it, and call the pandas DataFrame "Master".

In [ ]:
Master = pd.read_csv("../../baseballdatabank/core/Master.csv")   #read Master.csv 

type(Master)                                                  #show the python type for the result

In data science, most of the effort is spent exploring the data, discovering its characteristics, and massaging it into a form that you can apply modeling techniques to.

Jupyter is a great tool for this because you can make up the code piece-by-piece as you discover things about the data, rather than having to know in advance how you should process the data.

It's not realistic to expect that you will know enough about the data at the start to completely specify what you need to do to clean it up and make it ready for analysis.

We'll start with some really basic things like listing the column names (in this case, they were read from the first row in Master.csv).

In [ ]:
Master.columns               #list the column names of a DataFrame

One of the quickest ways to learn about the content of a DataFrame is to print a few rows.

The "head()" "tail()" methods allow you to do this without writing code.

In object oriented programming terminology, these are "methods" that the pandas.core.frame.DataFrame "class" provides.

To use them, you have to specify an "instance" of pandas.core.frame.DataFrame for them to operate on.

In this case, we want them to operate on the DataFrame we created from Master.csv, which we called "Master".

In [ ]:
Master.head()

This is our first look at the data.

One thing that jumps out is the "playerID" column, which seems to contain a kind of mish-mash of the player's name and a number.

It's a pretty good bet that this is some kind of unique identifier for that player.

Columns that contain unique identifiers play an important role in the "data wrangling" phase of the analysis, and we want to make sure pandas knows which columns have this property.

We make pandas aware by designating this column as an index.

Having learned something from our first peek at the data, let's go back and reread it but this time designate "playerID" as an index:

In [ ]:
Master = pd.read_csv("../../baseballdatabank/core/Master.csv",   #read Master.csv and designate "playerID" as an index
                     index_col=["playerID"])
type(Master)                                                  #show the python type for the result

In [ ]:
Master.head()

Although it looks like nothing changed, internally pandas now knows that the playerID column contains unique identifiers.

This will allow pandas to make intelligent decisions when we want to merge tables together.

You can see from the output of the "ls -l" command that we are working with a collection of files, and we will amlost certainly need to merge them together at some point.

These merge operations will rely on matching up the unique values in an index.

Our experience with playerID is a good illustration of why a Jupyter notebook is well-suited to data exploration.

We didn't know playerID was a good candidate for an index column until we had seen the data.

Once we had seen it, we decided to go back and redo the read_csv() sligthly differently.

This is pretty much the nature of "data munging", it's not a straight path from start to finish, but a series of discoveries that often lead us to back up and redo things.

## Exploring a DataFrame with describe()

Keep in mind pandas was designed from the ground up for "data wrangling", also known as "data_munging".

Usually this is where most of the work happens in any project that involves a lot of raw data.

Many people who have experience working with data say that as a rule of thumb, this will be 90% of the overal effort.

pandas includes a nice method called "describe()" designed to save time in the data exploration stage by automatically summarizing all of the numerical columns in a DataFrame.

In [ ]:
Master.describe()

The describe() method gives us mean, min, quartiles, max, standard deviation, and count.

count is the number of non-missing values.  Note that the counts vary quite a bit, indicating that this data has a lot of missing values.

Discovering what values are missing and deciding how to handle them is usually a major part of the data wrangling phase.

In some cases there will be a perfectly valid reason why a value is missing (deathYear for players who are alive).

In other cases, the data element may be missing because it is unavailable, or because it was recorded incorrectly, or because the source provided an invalid value to begin with.

Real-world data is almost always "messy" in this regard.  

It is very important that you be aware of what data is missing, which is something you usually have to discover for yourself early in the analysis.

## Dates, Times, and DateTimes

Very often the analysis we want to perform involves some kind of trend, which necessitates working with dates.

As you start working with dates, you will quickly discover that the standard "year,month,day" representation of dates is problematic.

To discover trends, we want to treat our data as a time series, and the Y-M-D representation does not allow us to easily compute the number of time units between two dates if our time unit is days or months.

Software designed for data manipulation almost always includes an internal representation of dates and times (and combined date-time values) that gives us a continuous scale.

For example, in SAS, a datatime value is the number of seconds since midnight on January 1, 1960.  There are many similar definitions in other software.

The advantage of this is that we can just subtract two datetime values and get the number of seconds in that interval, which we can then convert to other units like days (a day being 24 x 3600 or 86,400 seconds).  

Converting datetime intervals to months or years is more difficult because not all months are the same length, and there are leapyears. A 'leapyear' is a year with 366 days, the extra day being February 29th.  

Years that are multiples of 4 are leapyears, unless they are multiples of 100 and not multiples of 400.  So, 2000 was a leapyear but 1900 was not because 1900 is a multiple of 100, but not 400 and therefore an exception to the "multiples of 4" rule.  

Evidently, working with dates, times, and datetimes is complicated.  

As you would expect, pandas provides a comprehensive set of tools for working with dates and times.

You can find the full documentation here:

http://pandas.pydata.org/pandas-docs/stable/timeseries.html

This is a bit of overkill because for now we just want to explore some basic date processing.

One of the columns in our Master DataFrame, "finalGame", sounds like it would be a date, but as a single value it is probably in one of the commonly used external date formats.

There are a number of possibilities for this (YYYY-MM-DD,   YY-MM-DD,   MM-DD-YYYY,   MM-DD-YY,  etc.).

The easiest way to discover which one we have is to print a few values of Master.finalGame:

In [ ]:
Master.finalGame[1:10]

OK, it looks like the format is 'YYYY-MM-DD'.  

Notice that the type (dtype) of the finalGame column is "object".  

Unlike most other data analysis software, there is no requirement in pandas that the elements of a column all be of the same type.  

If you have more than one data type in a column, pandas uses the smallest class that can represent all of them, usually object.

Let's look at a nonmissing finalGame value and see what type it is:

In [ ]:
type(Master.finalGame[1])

This tells us that the dates are stored as strings of characters.  

Strings are just text and have no meaning beyond what you see.  

pandas is not aware that these strings represent dates.

pandas has a function for converting strings to pandas dates, but there is the potential for things to get complicated.

We could call this function for every value in the finalGame column, making a provision for handling the exception that will occur when we supply a missing value.

It's possible that there are other data types in the column, "object" could be anything.  If there are, we have to discover them and what they mean, so we need to spend more time understanding what exactly is in this column.

A much better approach is to exploit the facilities of the read_csv() function.  read_csv() has a provision for automatically recognizing dates in the input.

It will perform the necessary conversion for dates that are valid, and supply a missing value for those that are not.

Best of all, it will happen transparently when we call the read_csv() function, so we don't have to write code, just specify an option on the call to read_csv().

You can find the full documentation on read_csv() here:  (as usual, the full documentation is overkill for us)

http://pandas.pydata.org/pandas-docs/stable/generated/pandas.read_csv.html

The clean, simple solution to the very common problem of converting dates is an indication that the author of the pandas package, Wes McKinney, had a good understanding of the problems one encounters working with data, and did a good job designing software to solve them.

OK, let's read Master.csv a third time, but use the parse_dates option of read_csv() to tell pandas that the 'finalGame' column contains dates and we want to convert them.

In [ ]:
Master = pd.read_csv("../../baseballdatabank/core/Master.csv",   #read Master.csv and designate "playerID" as an index
                     index_col=["playerID"],parse_dates=['finalGame'])

In [ ]:
Master.finalGame[1:10]

Notice that the dtype for column 'finalGame' is no longer "object", but "datetime64[ns]", the internal pandas datetime data type.

This means the column is now a clean list of dates in the internal pandas datetime representation.

One thing worth noting here, even though the first entry displays as 1976-10-03 in both versions of "Master", the internal representations are entirely different.

When we display the finalGame column from the new "Master", pandas automatically recognizes that the 64-bit floating point datetime value will only make sense if it is displayed as a standard 'YYYY-MM-DD' string, but that is NOT what is stored in the new "Master".

pandas is now "aware" that these values represent dates.